<a href="https://colab.research.google.com/github/merveegokk/Energy-Consumption-Prediction/blob/main/Energy_Consumption_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
dataset_path = kagglehub.dataset_download(
    "jamshaid1202/smart-home-energy-consumption-and-weather-data"
)

import os
os.listdir(dataset_path)
from kagglehub import KaggleDatasetAdapter
import kagglehub

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "jamshaid1202/smart-home-energy-consumption-and-weather-data",
    "Smart Home Dataset.csv"
)

df.head()


100%|██████████| 19.4M/19.4M [00:00<00:00, 122MB/s] 

Extracting files...



/tmp/ipython-input-2297364193.py:13: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Download already complete (130953384 bytes).


/usr/local/lib/python3.12/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (0,27) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],...,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


Veri Seti Hakkında Bu veri seti, akıllı ev aletlerinden dakika dakika kaydedilen enerji tüketim verilerini (kilovat (kW) cinsinden) ve aynı bölgeden alınan hava durumu verilerini içerir. Veri seti, gerçek zamanlı akıllı sayaç okumalarını sıcaklık, nem ve diğer hava koşulları gibi çevresel faktörlerle birleştirerek enerji kullanım modellerini analiz etmek, ev enerji tüketimini optimize etmek ve havanın enerji talebi üzerindeki etkisini araştırmak için idealdir. Bu veri seti özellikle akıllı ev otomasyonu, enerji yönetim sistemleri ve IoT tabanlı enerji verimliliği çalışmaları için kullanışlıdır.


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503911 entries, 0 to 503910
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   time                 503911 non-null  object 
 1   use [kW]             503910 non-null  float64
 2   gen [kW]             503910 non-null  float64
 3   House overall [kW]   503910 non-null  float64
 4   Dishwasher [kW]      503910 non-null  float64
 5   Furnace 1 [kW]       503910 non-null  float64
 6   Furnace 2 [kW]       503910 non-null  float64
 7   Home office [kW]     503910 non-null  float64
 8   Fridge [kW]          503910 non-null  float64
 9   Wine cellar [kW]     503910 non-null  float64
 10  Garage door [kW]     503910 non-null  float64
 11  Kitchen 12 [kW]      503910 non-null  float64
 12  Kitchen 14 [kW]      503910 non-null  float64
 13  Kitchen 38 [kW]      503910 non-null  float64
 14  Barn [kW]            503910 non-null  float64
 15  Well [kW]        

In [7]:
#eksik veri olan satırları siliyorum.
df.dropna(inplace=True)

time,icon,summary,cloudCover kolonları object tipinde geri kalanı sayısal değerlerde. Modelin öğrenebilmesi için onları da anlayabilecekleri forma getiriyorum.                 

In [8]:
df["time"].head()

,time
0,1451624400
1,1451624401
2,1451624402
3,1451624403
4,1451624404


In [9]:
df['time'] = pd.to_datetime(df['time'], unit="s")
#saniye cinsinden girilmiş veriler o yüzden unit="s" ile
#saniyeye göre ayırıp gün ay saat diye güncelledim
df['hour'] = df['time'].dt.hour
df['day_of_week'] = df['time'].dt.dayofweek
df['month'] = df['time'].dt.month


/tmp/ipython-input-4123670149.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['time'] = pd.to_datetime(df['time'], unit="s")


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 503910 entries, 0 to 503909
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   time                 503910 non-null  datetime64[ns]
 1   use [kW]             503910 non-null  float64       
 2   gen [kW]             503910 non-null  float64       
 3   House overall [kW]   503910 non-null  float64       
 4   Dishwasher [kW]      503910 non-null  float64       
 5   Furnace 1 [kW]       503910 non-null  float64       
 6   Furnace 2 [kW]       503910 non-null  float64       
 7   Home office [kW]     503910 non-null  float64       
 8   Fridge [kW]          503910 non-null  float64       
 9   Wine cellar [kW]     503910 non-null  float64       
 10  Garage door [kW]     503910 non-null  float64       
 11  Kitchen 12 [kW]      503910 non-null  float64       
 12  Kitchen 14 [kW]      503910 non-null  float64       
 13  Kitchen 38 [kW]    

In [11]:
#modelde use,gen ve icon kolonlarını kullanmayacağım
df = df.drop(columns=['use [kW]', 'gen [kW]', 'icon'])

In [16]:
#cloudCover kolonunu da sayısal değere çeviriyorum böylece
#datasetindeki tüm veriler sayısal olucak.
df['cloudCover'] = pd.to_numeric(df['cloudCover'], errors='coerce')


Amaç:hava durumuna bakarak evin tüketeceği toplam enerjiyi tahmin etmek.Bu yüzden pivot kullanarak hava durumu bilgilerini gruplandırıp daha kolay anlaşılır forma getiriyorum

In [26]:
df_pivot = df.drop(columns=['time']).pivot_table(
    index='summary',
    aggfunc='mean')
#index='summary'ile Hava durumu durumlarına göre gruplara ayırdım
#time kolonunu dışarda tuttum çünkü onun veri tipi datetime
#sadece sayısal kolonları kullandım

In [27]:
df_pivot.head()

,Barn [kW],Dishwasher [kW],Fridge [kW],Furnace 1 [kW],Furnace 2 [kW],Garage door [kW],Home office [kW],House overall [kW],Kitchen 12 [kW],Kitchen 14 [kW],...,hour,humidity,month,precipIntensity,precipProbability,pressure,temperature,visibility,windBearing,windSpeed
summary,,,,,,,,,,,,,,,,,,,,,
Breezy,0.061821,0.000080,0.050323,0.070830,0.118513,0.013907,0.061340,0.615712,0.001784,0.008883,...,14.954516,0.412031,1.0,0.000056,0.001115,1008.677303,50.278065,9.948821,293.758488,19.930141
Breezy and Mostly Cloudy,0.035621,0.000271,0.029016,0.125615,0.264638,0.013076,0.042586,0.679730,0.000612,0.000117,...,15.000000,0.650000,1.0,0.002300,0.060000,1000.600000,41.630000,9.910000,284.000000,19.070000
Breezy and Partly Cloudy,0.033341,0.091140,0.063106,0.072180,0.100136,0.013182,0.070365,0.602535,0.000633,0.002774,...,15.399616,0.469039,1.0,0.000184,0.006129,1005.638818,50.873401,9.785495,302.185399,20.309693
Clear,0.059972,0.032208,0.063835,0.098767,0.137241,0.014205,0.083033,0.872838,0.002920,0.007148,...,11.805394,0.639287,1.0,0.000139,0.004214,1017.575511,50.309031,9.577906,206.907589,6.433823
Drizzle,0.054369,0.022395,0.065942,0.090118,0.119884,0.014883,0.069193,0.716580,0.000630,0.006447,...,10.747541,0.797878,1.0,0.006758,0.326885,1011.902854,54.615604,8.446555,165.403568,5.940559


In [32]:
#veriyi test ve train olarak ayırma:
# y: Evin toplam tükettiği enerji
y = df['House overall [kW]']

# X: modele vericeğim kolonlar
silinecekler = [
    'House overall [kW]', 'Dishwasher [kW]', 'Fridge [kW]',
    'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Garage door [kW]',
    'Home office [kW]', 'Kitchen 12 [kW]', 'Kitchen 14 [kW]',
    'Kitchen 38 [kW]', 'Barn [kW]', 'Well [kW]', 'Microwave [kW]',
    'Living room [kW]', 'Solar [kW]', 'summary'
]
#bu kolonları siliyorum çünkü House overall [kW] kolonu bu bilgiler sahip
#model kopya çekmesin diye onları kaldırıyorum geri kalanlarda X oluyor.

temiz_df = df.drop(columns=silinecekler)
X = temiz_df

In [33]:
X=X.drop(columns=['time'])

In [37]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 503910 entries, 0 to 503909
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Wine cellar [kW]     503910 non-null  float64
 1   temperature          503910 non-null  float64
 2   humidity             503910 non-null  float64
 3   visibility           503910 non-null  float64
 4   apparentTemperature  503910 non-null  float64
 5   pressure             503910 non-null  float64
 6   windSpeed            503910 non-null  float64
 7   cloudCover           503852 non-null  float64
 8   windBearing          503910 non-null  float64
 9   precipIntensity      503910 non-null  float64
 10  dewPoint             503910 non-null  float64
 11  precipProbability    503910 non-null  float64
 12  hour                 503910 non-null  int32  
 13  day_of_week          503910 non-null  int32  
 14  month                503910 non-null  int32  
dtypes: float64(12), int32(

In [38]:
X=X.drop(columns=['Wine cellar [kW]'])

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [41]:
# random forest ile eğitim:
rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
print("Random Forest Başarı Sonuçları:")
print(f"R2 Skoru: {r2_score(y_test, y_pred_rf):.4f}")
print(f"MAE (Hata): {mean_absolute_error(y_test, y_pred_rf):.4f} kW")

Random Forest Başarı Sonuçları:
R2 Skoru: 0.5563
MAE (Hata): 0.3438 kW


In [47]:
# Linear Regression ile eğitim:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

print("Linear Regression Sonuçları:")
print(f"R2 Skoru (Başarı): {r2_score(y_test, y_pred_lr):.4f}")
print(f"MAE (Hata): {mean_absolute_error(y_test, y_pred_lr):.4f} kW")

Linear Regression Sonuçları:
R2 Skoru (Başarı): 0.0203
MAE (Hata): 0.5762 kW


In [48]:
# Decision Tree Regressor Modeli ile eğitim
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

y_pred_dt = dt_model.predict(X_test)

print("Decision Tree Sonuçları")
print(f"R2 Skoru (Başarı): {r2_score(y_test, y_pred_dt):.4f}")
print(f"MAE (Hata): {mean_absolute_error(y_test, y_pred_dt):.4f} kW")

Decision Tree Sonuçları
R2 Skoru (Başarı): 0.5567
MAE (Hata): 0.3436 kW
